In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv("../input_data/intrusion_detection/data.csv")
Y = df.xAttack
X = df.drop(['xAttack'],axis=1)
X = (X - X.mean())/X.std()
X.head()

,duration,service,src_bytes,dst_bytes,hot,num_failed_logins,num_compromised,num_root,num_file_creations,num_access_files,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,-0.114536,-0.425928,-0.010012,-0.034506,-0.090954,-0.026321,-0.021938,-0.021801,-0.027915,-0.044086,...,-1.690179,1.262846,1.067550,-0.441075,-0.253420,0.071029,-0.639487,-0.535321,-0.386069,-0.374975
1,-0.114536,0.362284,-0.010092,-0.039818,-0.090954,-0.026321,-0.021938,-0.021801,-0.027915,-0.044086,...,0.732932,-1.032492,-1.159522,-0.069608,-0.479712,-0.289000,-0.639487,-0.625133,2.878183,2.771083
2,-0.114536,-0.425928,-0.010023,0.077314,-0.090954,-0.026321,-0.021938,-0.021801,-0.027915,-0.044086,...,-1.447868,1.262846,1.067550,-0.441075,-0.382730,0.251043,-0.639487,-0.625133,-0.386069,-0.374975
3,-0.114536,-0.729086,-0.009539,-0.039818,-0.090954,-0.026321,-0.021938,-0.021801,-0.027915,-0.044086,...,-0.488720,-0.761389,-0.647295,-0.228808,0.263818,-0.289000,-0.594495,-0.625133,-0.386069,-0.374975
4,-0.114536,0.301652,-0.010092,-0.039818,-0.090954,-0.026321,-0.021938,-0.021801,-0.027915,-0.044086,...,0.732932,-0.815609,-0.936814,-0.175741,-0.479712,-0.289000,1.610076,1.620172,-0.386069,-0.374975


In [3]:
pca = PCA(.90)
principalComponents = pca.fit(X)
X_red = pca.transform(X)
X_new = pd.DataFrame( X_red,columns=[ "pc"+str(i) for i in xrange(X_red.shape[1]) ] )

In [4]:
finalDf = pd.concat([X_new, df[['xAttack']]], axis = 1)

In [5]:
pca.explained_variance_ratio_

array([0.23233278, 0.16771547, 0.09010134, 0.06510947, 0.05077007,
       0.04635866, 0.03920077, 0.03641207, 0.03469033, 0.03440369,
       0.03320159, 0.03168948, 0.02472306, 0.02261023])

In [6]:
cov_x = np.cov(X.T)
cov_x.shape

(29, 29)

In [7]:
U,S,V = np.linalg.svd(cov_x)
S_sum = float(np.sum(S))

In [8]:
running_sum = 0
num_of_comp = 0
for i in xrange(len(S)):
    running_sum += S[i]
    if running_sum  / S_sum  >= 0.90:
        num_of_comp = i+1
        break
print num_of_comp

14


In [9]:
print U.shape

(29, 29)


In [10]:
U_red = U[:,:num_of_comp]
print U_red.shape

(29, 14)


In [11]:
Z = np.matmul(U_red.T, X.T)
Z = Z.T
Z_new = pd.DataFrame( Z,columns=[ "pc"+str(i) for i in xrange(Z.shape[1]) ] )

In [12]:
X_new.head()

,pc0,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,pc11,pc12,pc13
0,-2.509415,-0.952099,-0.080574,0.089473,-0.465558,-0.264131,-0.073175,0.122966,0.115928,-0.037054,0.015485,0.085030,0.478752,-0.380050
1,1.193236,5.598092,-0.270790,1.130941,-0.813088,-1.629181,0.541842,-0.190282,-0.243673,0.079391,-0.028093,0.064366,-0.430109,-0.264010
2,-2.447051,-0.908922,-0.060318,0.031070,-0.822722,-0.170780,-0.098459,0.161764,0.088158,0.084797,-0.005994,0.050481,0.754346,-0.429607
3,-1.111736,-0.343088,0.004162,-0.266447,-0.166211,0.596454,-0.015911,-0.026994,-0.161910,0.061835,-0.050150,-0.362922,0.542284,-0.408395
4,3.983887,-1.268837,-0.050827,0.252373,0.537136,-0.561597,0.278500,-0.038172,-0.012687,0.004357,0.013466,0.041690,-0.072666,0.071331


In [13]:
Z_new.head()

,pc0,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,pc11,pc12,pc13
0,-2.509415,-0.952099,-0.080574,0.089473,-0.465558,0.264131,-0.073175,-0.122966,-0.115928,0.037054,-0.015485,0.085030,0.478752,0.380050
1,1.193236,5.598092,-0.270790,1.130941,-0.813088,1.629181,0.541842,0.190282,0.243673,-0.079391,0.028093,0.064366,-0.430109,0.264010
2,-2.447051,-0.908922,-0.060318,0.031070,-0.822722,0.170780,-0.098459,-0.161764,-0.088158,-0.084797,0.005994,0.050481,0.754346,0.429607
3,-1.111736,-0.343088,0.004162,-0.266447,-0.166211,-0.596454,-0.015911,0.026994,0.161910,-0.061835,0.050150,-0.362922,0.542284,0.408395
4,3.983887,-1.268837,-0.050827,0.252373,0.537136,0.561597,0.278500,0.038172,0.012687,-0.004357,-0.013466,0.041690,-0.072666,-0.071331
